In [1]:
import pandas as pd
import numpy as np
import string
import ray
UPPER_CASE = string.ascii_uppercase

In [2]:
def generate_vals(n):
    return np.random.randint(low = -n, high = n, size = n)

def generate_df(p = 20, size= 10000):
    letters = np.random.choice(list(UPPER_CASE), size= size)
    df = pd.DataFrame({'labels': letters})
    for i in range(p):
        df['vals'+str(i)] = generate_vals(size)
    return df

def list_of_df(n = 10, p = 20, size=10000):
    return [generate_df(p = p, size=size) for i in range(n)]

In [3]:
list_df = list_of_df(n=20, p=50, size=100000)

### Appending to master data frames, or concatenating all at once

In [4]:
def append_df():
    columns = ['labels'] + ['val'+str(i) for i in range(20)]
    base = pd.DataFrame(columns=columns)
    for df in list_df:
        base.append(df)
    return base

def concat_df():
    return pd.concat(list_df)

In [5]:
%time dfr = append_df() 

CPU times: user 1.54 s, sys: 1.02 s, total: 2.56 s
Wall time: 2.56 s


In [6]:
%time dfr = concat_df() 

CPU times: user 307 ms, sys: 247 ms, total: 553 ms
Wall time: 553 ms


### Generate Date strings

In [22]:
def gen_date_str():
    year = str(np.random.randint(1900,2020))
    mon = str(np.random.randint(1,12)).zfill(2)
    day = str(np.random.randint(1,29)).zfill(2)
    return year + mon + day

def gen_rand_date(n):
    return [gen_date_str() for i in range(n)]


def gen_date_str_vec(n):
    year = pd.Series(np.random.randint(1900,2020, size=n)).astype(str)
    mon = pd.Series(np.random.randint(1,12, size=n)).astype(str).str.zfill(2)
    day = pd.Series(np.random.randint(1,29, size=n)).astype(str).str.zfill(2)
    return np.add(np.add(year,mon),day)


n = 1000000


In [23]:
%time res = gen_rand_date(n)

CPU times: user 6.54 s, sys: 46.4 ms, total: 6.58 s
Wall time: 6.59 s


In [24]:
%time res = gen_date_str_vec(n)

CPU times: user 7.57 s, sys: 154 ms, total: 7.72 s
Wall time: 7.72 s


In [25]:
date_df = pd.DataFrame({'label': np.random.choice(list(UPPER_CASE), size= n), 'date_str': gen_rand_date(n) })

### String Accessor methods vs. row by row : SAME

In [264]:
%time date_df['year'] = date_df['date_str'].map(lambda x : x[:4])

CPU times: user 304 ms, sys: 31 ms, total: 335 ms
Wall time: 335 ms


In [265]:
%time date_df['year'] = date_df['date_str'].str[:4]

CPU times: user 407 ms, sys: 60.3 ms, total: 467 ms
Wall time: 499 ms


### Comparison of parsing dates, row by row vs. vector : SAME

In [266]:
import datetime

In [267]:
%time date_df['date'] = date_df['date_str'].map(lambda x : datetime.date(int(x[:4]),int(x[4:6]),int(x[6:8])))

CPU times: user 1.44 s, sys: 28.7 ms, total: 1.47 s
Wall time: 1.49 s


In [268]:
def vector_date_conv():
    date_df['year'] = date_df['date_str'].str[:4]    
    date_df['day'] = date_df['date_str'].str[6:8]
    date_df['mon'] = date_df['date_str'].str[4:6]
    date_df['format_date'] = date_df['year'] + '-' + date_df['mon'] + '-' + date_df['day']
    date_df['date'] = date_df['format_date'].astype('datetime64[ns]')
    
%time vector_date_conv()

CPU times: user 1.51 s, sys: 225 ms, total: 1.73 s
Wall time: 1.75 s


### Use the accessor methods for date `.dt` : USE ACCESSORS

In [269]:
%time date_df['dow'] = date_df['date'].map(lambda x: x.dayofweek)

CPU times: user 3.97 s, sys: 364 ms, total: 4.34 s
Wall time: 4.48 s


In [270]:
%time date_df['dow'] = date_df['date'].dt.dayofweek

CPU times: user 99.5 ms, sys: 9.19 ms, total: 109 ms
Wall time: 122 ms


### Use `nsmallest` instead of sort + tail

In [271]:
%time dfr['vals0'].sort_values().head(5)

CPU times: user 356 ms, sys: 34 ms, total: 390 ms
Wall time: 388 ms


12249   -100000
51213   -100000
553     -100000
81483   -100000
158     -100000
Name: vals0, dtype: int64

In [272]:
%time dfr['vals0'].nsmallest(5)

CPU times: user 26.3 ms, sys: 6.47 ms, total: 32.7 ms
Wall time: 29.9 ms


12249   -100000
51213   -100000
81483   -100000
82713   -100000
158     -100000
Name: vals0, dtype: int64

### Passing a function row by row, or vectorize : VECTORIZE

In [273]:
dfr.shape

(2000000, 51)

In [287]:
def mult_row_vals(x):
    return (x['vals5']*np.log(abs(x['vals0'])+100) + x['vals15']*np.log(abs(x['vals10'])+100))*25

def row_approach(dfr):
    return dfr.apply(mult_row_vals, axis=1)

%time res = row_approach(dfr.head(100000))

CPU times: user 4.69 s, sys: 138 ms, total: 4.83 s
Wall time: 4.83 s


In [288]:
res[:5]

0    5.220946e+07
1    8.106892e+06
2    1.055659e+07
3    4.318175e+07
4   -6.942590e+06
dtype: float64

In [289]:
def vect_approach(dfr):
    return (dfr['vals5']*np.log(np.abs(dfr['vals0'])+100) + dfr['vals15']*np.log(np.abs(dfr['vals0'])+100))*25

%time res = vect_approach(dfr.head(100000)) 

CPU times: user 8.56 ms, sys: 6.06 ms, total: 14.6 ms
Wall time: 9.03 ms


In [290]:
res[:5]

0    5.152457e+07
1    7.491559e+06
2    1.169490e+07
3    4.212103e+07
4   -6.932527e+06
dtype: float64

### Concatentating Strings : - USE VECTORS

In [291]:
def generate_rand_letters(n):
    return np.random.choice(list(UPPER_CASE), size= n)

def generate_letter_matrix(p, n):
    df = pd.DataFrame({'letter0': generate_rand_letters(n)})
    for i in range(p):
        df['letter'+str(i)] = generate_rand_letters(n)
    return df

In [292]:
p = 15
n = 1000000
letter_df = generate_letter_matrix(p,n )

In [293]:
letter_df.head(10)

,letter0,letter1,letter2,letter3,letter4,letter5,letter6,letter7,letter8,letter9,letter10,letter11,letter12,letter13,letter14
0,W,M,Y,C,Q,H,G,C,M,V,I,D,A,I,K
1,C,K,I,J,V,Y,W,R,W,M,M,N,T,F,I
2,Q,F,N,S,Y,R,E,I,A,T,Z,M,N,H,S
3,C,D,C,V,N,H,D,N,Y,N,E,P,Z,D,M
4,X,G,B,P,A,F,N,H,F,A,E,U,M,J,D
5,B,P,F,M,S,O,O,R,B,L,F,N,W,L,W
6,N,W,Q,T,D,R,C,J,W,A,X,N,W,N,J
7,F,C,F,E,I,V,Q,D,C,N,H,Z,I,K,J
8,M,W,P,B,Z,P,U,M,I,T,U,Z,L,D,O
9,S,H,O,W,Y,B,T,S,A,C,W,H,T,K,M


In [312]:
def row_concat(x):
    return ''.join(x.values)

def row_wise_concat(df):
    res = letter_df.apply(row_concat, axis=1)
    return res

def vect_concat(df):
    res = df[df.columns[0]]
    for col in df.columns[1:]:
        res = res + df[col]
        
def vect_concat_np(df):
    res = df[df.columns[0]]
    for col in df.columns[1:]:
        res = np.add(res, df[col])

In [313]:
%time res = row_wise_concat(letter_df)

CPU times: user 6.96 s, sys: 102 ms, total: 7.06 s
Wall time: 7.07 s


In [314]:
%time res = vect_concat(letter_df)

CPU times: user 1.25 s, sys: 412 ms, total: 1.66 s
Wall time: 1.66 s


In [315]:
%time res = vect_concat_np(letter_df)

CPU times: user 1.2 s, sys: 337 ms, total: 1.54 s
Wall time: 1.54 s


### how to do vectorized Booleans

In [91]:
def vector_bool(dfr):
    dfr.loc[dfr['vals0'] > 0 ,'bools'] = 'greater'
    dfr.loc[(dfr['vals0'] < 0) & (dfr['vals0'] > -200) ,'bools'] = 'middle'
    dfr.loc[dfr['vals0'] <= -200 ,'bools'] = 'lesser'
    
%time vector_bool(dfr)
dfr['bools'].value_counts()

CPU times: user 162 ms, sys: 23.1 ms, total: 185 ms
Wall time: 183 ms


greater    999880
lesser     998163
middle       1957
Name: bools, dtype: int64

In [92]:
def row_bool(dfr):
    dfr['bools'] = dfr['vals0'].map(lambda x : 'greater' if x > 0 else 'middle' if x > -200 else 'lesser')
    
%time row_bool(dfr)
dfr['bools'].value_counts()

CPU times: user 463 ms, sys: 30.6 ms, total: 493 ms
Wall time: 491 ms


greater    999880
lesser     998163
middle       1957
Name: bools, dtype: int64

### Category Encoding - use category accessor

In [27]:
def generate_df2(p = 4, size= 5000000):
    letters = np.random.choice(list(UPPER_CASE), size= size)
    letters2 = np.random.choice(list(UPPER_CASE), size= size)
    print()
    letters3 = np.core.defchararray.add(letters, letters2)
    df = pd.DataFrame({'labels': letters3})
    for i in range(p):
        df['vals'+str(i)] = generate_vals(size)
    return df

dfr2 = generate_df2()

print(dfr.shape)
dfr.head()


(2000000, 51)


,labels,vals0,vals1,vals2,vals3,vals4,vals5,vals6,vals7,vals8,...,vals40,vals41,vals42,vals43,vals44,vals45,vals46,vals47,vals48,vals49
0,G,84697,-72535,-79252,66810,-31657,79770,-10382,83983,-43936,...,54208,45397,51916,76551,-60219,2483,-58132,29829,64332,-41015
1,E,66406,-76606,65406,-76517,8948,-7719,22714,-66707,83393,...,-42096,-44625,-99786,35159,5530,-52786,-18621,86401,71263,-80929
2,E,-25024,40538,67674,-58861,-54072,21152,78808,61342,41094,...,-11527,3601,-48855,98523,83312,21122,-27876,-15545,-89940,18302
3,E,-12245,76198,-83930,36266,-13695,67471,-33862,81932,32315,...,-92763,45019,-92581,-10273,-64823,-20734,-45269,-62873,39364,34986
4,F,69620,-33704,-44604,-27128,12023,-38341,-69285,-24263,-56125,...,-19291,-99282,-69234,35442,-41397,80520,38994,-91468,96107,-13626


In [34]:
def encode_cats(dfr):
    lkup = {v:i for i,v in enumerate(dfr['labels'].unique())}
    return(dfr2['labels'].map(lkup))

%time enc_col = encode_cats(dfr)

CPU times: user 594 ms, sys: 59.6 ms, total: 654 ms
Wall time: 654 ms


In [35]:
%time enc_col = dfr2['labels'].astype('category').cat.codes

CPU times: user 247 ms, sys: 47.3 ms, total: 294 ms
Wall time: 292 ms


### What about Numba? - Group by Example

In [30]:
import numba
from collections import defaultdict
import datetime

In [36]:
dfr2.head()

,labels,vals0,vals1,vals2,vals3
0,GJ,-1342885,-3092600,4632122,-4310428
1,BD,-4085191,-276194,-937774,-3396087
2,DN,2705057,-2136860,4743671,-364644
3,IQ,-4749400,3746472,-4259509,555150
4,KP,-2214651,1551929,1889357,2656194


In [38]:
%time res = dfr2.groupby('labels')['vals0'].sum()

CPU times: user 383 ms, sys: 182 ms, total: 565 ms
Wall time: 564 ms


In [39]:
def groupby_python(index, value, output):
    for i in range(index.shape[0]):
        output[index[i]] += value[i]
        
groupby_numba = numba.jit(groupby_python, nopython=True)

m_numba = np.zeros_like(res)

dfr2_c = dfr2.copy()
dfr2_c['labels'] = dfr2_c['labels'].astype('category').cat.codes 

%time groupby_numba(np.array(dfr2_c['labels'].values), np.array(dfr2_c['vals0'].values), m_numba)

CPU times: user 240 ms, sys: 62.3 ms, total: 302 ms
Wall time: 317 ms


### What about Numba? Row Sum Example

In [45]:
def generate_df(p = 20, size= 10000):
    letters = np.random.choice(list(UPPER_CASE), size= size)
    df = pd.DataFrame({'vals': generate_vals(size)})
    for i in range(p):
        df['vals'+str(i)] = generate_vals(size)
    return df

dfr_nums = generate_df()
dfr_nums.head()

,vals,vals0,vals1,vals2,vals3,vals4,vals5,vals6,vals7,vals8,...,vals10,vals11,vals12,vals13,vals14,vals15,vals16,vals17,vals18,vals19
0,3862,7611,-8525,-4328,5113,5135,6319,6406,-1455,8028,...,-1698,5461,-5913,8405,6067,1198,-6378,5147,-9182,8957
1,6325,7601,-3601,6364,-8801,-7859,607,-1990,-7881,-2241,...,6174,-695,-4420,-6247,-6095,9429,5414,-3339,7779,-4534
2,-847,-8953,112,-8245,-7096,-9238,-55,-9273,5156,-7337,...,-3351,6037,2812,1336,350,-7260,2680,-168,1824,-6986
3,6306,-2072,-2165,1156,-5262,-7942,-5729,-6968,4574,-426,...,2039,-4383,7946,-4554,-2215,7018,3073,-2190,7119,-9152
4,-3031,739,-5406,4266,-6549,7769,4829,-259,-9579,-6760,...,5068,-8643,-7649,-5893,-4074,-679,-2980,-9623,7041,4583


In [56]:
%time res = dfr_nums.sum(axis=1)

CPU times: user 2.22 ms, sys: 1.34 ms, total: 3.56 ms
Wall time: 1.72 ms


In [63]:
def rowsums_python(matrix, output):
    for i in range(matrix.shape[0]):
        row_sum = 0
        for j in range(matrix.shape[1]):
            row_sum += matrix[i,j]
        output[i] = row_sum
    return output

rowsums_numba = numba.jit(rowsums_python, nopython=True)

mtrx = dfr_nums.as_matrix()
m_numba = np.zeros(mtrx.shape[0])

%time res = rowsums_numba(mtrx, m_numba)


CPU times: user 135 ms, sys: 3.56 ms, total: 139 ms
Wall time: 137 ms


### Vectorized Numba Add

In [69]:
x = np.random.random(1000000)
y = np.random.random(1000000)

In [72]:
%time x+y

CPU times: user 3.08 ms, sys: 1.68 ms, total: 4.77 ms
Wall time: 3.11 ms


array([ 0.4723531 ,  0.84313548,  0.64643165, ...,  1.45630092,
        0.54166015,  0.22555774])

In [75]:
from numba import vectorize
@vectorize
def numba_add(x,y):
    return x + y

%time numba_add(x,y)
    

CPU times: user 70.8 ms, sys: 4.39 ms, total: 75.2 ms
Wall time: 73.9 ms


array([ 0.4723531 ,  0.84313548,  0.64643165, ...,  1.45630092,
        0.54166015,  0.22555774])